# P1.1: Building an autoregressive language model (Simplified GPT-1)

Hey! In this part, you will be building a simple autoregressive model to predict the next word based on the previous words. Please run the code blocks below and do not modify anything except the TODO and MODIFIABLE parts.

In [ ]:
### IF YOU ARE USING COLAB, UNCOMMENT AND RUN THIS BLOCK FIRST ###
# # Please edit the path below to be the assignment3 folder in your Google Drive
# !pip install -r /content/drive/MyDrive/cs589/assignment3/requirements.txt

## Import the libraries

In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from nltk.tokenize import word_tokenize
import numpy as np
import nltk

nltk.download('punkt')

## Import the dataset

We will be importing a dataset of multiple different student essays on various topics from Hugging Face. Here is the [source](https://huggingface.co/datasets/deepachalapathi/essay_grade_v1) if you are interested!

In [ ]:
# Load human text dataset
splits = {'train': 'data/train-00000-of-00001-c7efd4234b769731.parquet', 'validation': 'data/validation-00000-of-00001-160da3cc6fc122a4.parquet'}
df = pd.read_parquet("hf://datasets/deepachalapathi/essay_grade_v1/" + splits["train"])

In [ ]:
n = 10 # MODIFIABLE: This slices the dataset down to only n rows. Change this if you want to optimize your model, but it will take longer. The total number of rows is 1467
sliced_df = df.iloc[:n]
corpus = sliced_df['text'].tolist()
print(corpus[0])

## Defining classes and utilities functions

In [ ]:
class TextDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx][0]), torch.tensor(self.data[idx][1])

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        output, (hidden, _) = self.lstm(x)
        output = self.fc(output[:, -1, :])
        return output

In [ ]:
def plot_curve(loss_history):
    plt.plot(np.arange(len(loss_history)), loss_history, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.show()

## Build the language model

This language model is an autoregressive model that predicts the next word based on the k words coming before it. In the code below, `k` would be represented as `window_size`.

Essentially, for every word $x_i$, we are computing the probability it appears given the previous words.

$$
p(x) = p(x_1)P(x_2|x_1)...p(x_i|x_{<i})
$$

We are computing this by assigning the preceding words as training data and the word itself as label, implementing a LSTM-based neural network that minimizes the cross-entropy loss between the predicted label with the actual label and updates the probability sequentially.

Please fill in the TODOs part below to complete building the language model

In [ ]:
class LanguageModel:
    def __init__(self, corpus, window_size=2):
        self.corpus = corpus
        self.window_size = window_size
        self.vocab = {}
        self.vocab_size = 0
        self.model = None
        self.tokenizer = nltk.download('punkt')
        self._prepare_data()

    def _prepare_data(self):
        # Tokenize the text to arrays of sentences, which are arrays of lowercased words
        corpus_tokens = [word_tokenize(doc.lower()) for doc in self.corpus]

        # TODO: Assign each unique words a number inside self.vocab (key: token, value: number)
        # --- YOUR CODE STARTS HERE --- #

        # --- YOUR CODE ENDS HERE --- #

        # Create sequences of window_size
        sequences = []
        for sentence in corpus_tokens:
            sequences.append([self.vocab[token] for token in sentence])

        # Create a dataset with y being the next word after a sequence of window of words of size k
        data = []
        for sequence in sequences:
            for i in range(len(sequence)):
                X = sequence[max(i - self.window_size + 1, 0):i]
                y = sequence[i]
                # Pad sequences so that there is no errors training on the first few words, where there are no preceding words
                X = [0]*(self.window_size - len(X)) + X
                data.append((X, y))

        self.dataset = TextDataset(data)
        self.dataloader = DataLoader(self.dataset, batch_size=2, shuffle=True)

    def build_model(self, embed_dim=10, hidden_dim=50):
        print("Building the model...")
        self.model = LSTMModel(self.vocab_size, embed_dim, hidden_dim)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def train(self, epochs=50):
        print("Training the model...")
        loss_history = []
        # Update the model with mini-batches
        for epoch in range(epochs):
            for inputs, labels in self.dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
                
            loss_history.append(loss.item())
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss.item()}')
        return loss_history

    def predict_next_word(self, text):
        self.model.eval()
        words = word_tokenize(text.lower())
        indices = [self.vocab[word] if word in self.vocab else 0 for word in words]
        indices = [0]*(self.window_size - len(indices)) + indices  # Padding
        indices = torch.tensor([indices])
        with torch.no_grad():
            outputs = self.model(indices)
            predicted_index = outputs.argmax(1).item()
        reverse_vocab = {index: word for word, index in self.vocab.items()}
        return reverse_vocab[predicted_index]

    def build_sentence(self, text, word_count=10):
        # TODO: Utilize the predict_next_word() function to build sentence
        # --- YOUR CODE STARTS HERE --- #

        # --- YOUR CODE ENDS HERE --- #

## Training models

Let's create multiple language models with different window size and compare them! The training time for each model should take about 2 minutes. Depending on the size of data defined above, the window size, and the number of epochs, the training time can take a lot longer. We recommend you increase the number of epochs or the data size if you are looking for better results. 

Click "Run All" and go do something on your own since it will take a while. Watch your favorite show, take a shower or touch some grass :>

#### window_size = 2

In [ ]:
lm_k2 = LanguageModel(corpus, window_size=2)
lm_k2.build_model()
loss_history_k2 = lm_k2.train(epochs=50)
plot_curve(loss_history_k2)

#### window_size = 3

In [ ]:
lm_k3 = LanguageModel(corpus, window_size=3)
lm_k3.build_model()
loss_history_k3 = lm_k3.train(epochs=50)
plot_curve(loss_history_k3)

### window_size = 4

In [ ]:
lm_k4 = LanguageModel(corpus, window_size=2)
lm_k4.build_model()
loss_history_k4 = lm_k4.train(epochs=50)
plot_curve(loss_history_k4)

### Let's have fun!

Now that we have trained our model, time to test them out with your own incomplete sentence! Go all out and type something below. Let's see how the three models complete it!

In [ ]:
# TODO: Set up the config below
sentence_prefix = ""
sentence_size = 10
generated_size = sentence_size - len(sentence_prefix.split())

#### window_size = 2

In [ ]:
result_k2 = lm_k2.build_sentence(sentence_prefix, generated_size)
print(result_k2)

#### window_size = 3

In [ ]:
result_k3 = lm_k3.build_sentence(sentence_prefix, generated_size)
print(result_k3)

#### window_size = 4

In [ ]:
result_k4 = lm_k4.build_sentence(sentence_prefix, generated_size)
print(result_k4)

## Evaluation

Compare the result between each model of different window sizes. How does the window size affect the results? Can you explain why it affects the results that way?

*Your answer:*

## Go wild (Extra credit)

Be creative and try different configurations! Try increasing the window size or the data size even more and tell us what you observe. You can try with different prompts too and see how that affects the result!

In [ ]:
### YOUR CODE HERE ###

*Your observation here:*

# P1.2 Free Response

**Q1**: Let's say the CICS department is conducting research on predicting applicant rate for the next 5 years. The data on applicant rate for the last many years is available for you train. Which deep generative model would you choose for this task? Explain how the model works and how it would be applied to this problem.

*Your answer here:*

**Q2**: UMass is interested in generating images of future campus buildings from blueprints. Explain how diffusion models could assist in this task and outline the steps involved in training such a model.

*Your answer here:*

**Q3**: The campus security team at UMass wants to use Generative Adversarial Networks (GANs) to enhance their surveillance system by generating realistic scenarios for training purposes. What would be the roles of the generator and the discriminator in this context?

*Your answer here:*

**Q4**: In the process of training our logistic regression classifier with generative model, we have to find the right set of parameters $\theta$ to minimize the KL divergence as in the formula below.

$$
L(\theta) = KL(p_{\text{data}} \| p_\theta) = \mathbb{E}_{x \sim p_{\text{data}}}\left[\log \frac{p_{\text{data}}(x)}{p_\theta(x)}\right]
$$

However, in practice, we don't have direct access to the true data distribution $p_{\text{data}}$. Which technique can we use to estimate this?

*Your answer here:*

**Q5:** What is a problem that you are having right now that you think you can solve with generative modeling? ("My problem is this homework and a generative model like ChatGPT will help me solve this" won't work. Think of something outside school.) Describe which model and technique you will be using and why you use them. Will there be any potential problem?

*Your answer here:*